In [1]:
# The main difference in the api's is how you import the models and evaluate performance
# These api's can be found under ampligraph.compat APIs
# from ampligraph.compat import TransE, DistMult, ComplEx, HolE, evaluate_performance

In [2]:
import sys
sys.path.append('../..')
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
import numpy as np

In [3]:
import ampligraph
# load the dataset
from ampligraph.datasets import load_fb15k_237
X = load_fb15k_237()

In [4]:

# import the models from ampligraph.compat
# we support only TransE, DistMult, ComplEx, HolE
from ampligraph.compat import ComplEx

model = ComplEx(batches_count=10, seed=0, epochs=2000, k=350, eta=10,
                    # Use adam optimizer with learning rate 1e-3
                    optimizer='adam', optimizer_params={'lr':1e-3},
                    # Use multiclass_nll loss 
                    loss='multiclass_nll', loss_params={},
                    # Use L3 regularizer with regularizer weight 1e-3
                    regularizer='LP', regularizer_params={'p':3, 'lambda':1e-3}, 
                    # Enable stdout messages (set to false if you don't want to display)
                    verbose=True)


In [5]:
# create the filter
filter = np.concatenate((X['train'], X['valid'], X['test']))


In [6]:
# Fit the model on training and validation set
model.fit(X['train'], 
          early_stopping = True,
          early_stopping_params = \
                  {
                      'x_valid': X['valid'][::10],       # validation set
                      'criteria':'hits@10',         # Uses hits10 criteria for early stopping
                      'burn_in': 20,              # early stopping kicks in after 100 epochs
                      'check_interval':20,         # validates every 20th epoch
                      'stop_interval':5,           # stops if 5 successive validation checks are bad.
                      'x_filter': filter,          # Use filter for filtering out positives 
                      'corruption_entities':'all', # corrupt using all entities
                      'corrupt_side':'s+o'         # corrupt subject and object (but not at once)
                  }
          )

Epoch 1/2000
11/11 [==============================] - 2s 224ms/step - loss: 65250.1641
Epoch 2/2000
11/11 [==============================] - 1s 83ms/step - loss: 65240.8359
Epoch 3/2000
11/11 [==============================] - 1s 83ms/step - loss: 65222.4375
Epoch 4/2000
11/11 [==============================] - 1s 83ms/step - loss: 65179.2812
Epoch 5/2000
11/11 [==============================] - 1s 82ms/step - loss: 65083.1094
Epoch 6/2000
11/11 [==============================] - 1s 81ms/step - loss: 64889.0352
Epoch 7/2000
11/11 [==============================] - 1s 81ms/step - loss: 64534.9219
Epoch 8/2000
11/11 [==============================] - 1s 84ms/step - loss: 63944.8438
Epoch 9/2000
11/11 [==============================] - 1s 81ms/step - loss: 63039.1602
Epoch 10/2000
11/11 [==============================] - 1s 84ms/step - loss: 61753.2969
Epoch 11/2000
11/11 [==============================] - 1s 82ms/step - loss: 60068.1680
Epoch 12/2000
11/11 [==============================

11/11 [==============================] - 1s 78ms/step - loss: 12602.2197
Epoch 89/2000
11/11 [==============================] - 1s 80ms/step - loss: 12490.2334
Epoch 90/2000
11/11 [==============================] - 1s 81ms/step - loss: 12380.6807
Epoch 91/2000
11/11 [==============================] - 1s 80ms/step - loss: 12273.6699
Epoch 92/2000
11/11 [==============================] - 1s 79ms/step - loss: 12168.8193
Epoch 93/2000
11/11 [==============================] - 1s 79ms/step - loss: 12065.9434
Epoch 94/2000
11/11 [==============================] - 1s 79ms/step - loss: 11965.1338
Epoch 95/2000
11/11 [==============================] - 1s 80ms/step - loss: 11866.1641
Epoch 96/2000
11/11 [==============================] - 1s 78ms/step - loss: 11769.2451
Epoch 97/2000
11/11 [==============================] - 1s 80ms/step - loss: 11674.4580
Epoch 98/2000
11/11 [==============================] - 1s 88ms/step - loss: 11581.3438
Epoch 99/2000
11/11 [==============================] - 1s

11/11 [==============================] - 1s 82ms/step - loss: 7545.4492
Epoch 175/2000
11/11 [==============================] - 1s 79ms/step - loss: 7514.9331
Epoch 176/2000
11/11 [==============================] - 1s 78ms/step - loss: 7484.8115
Epoch 177/2000
11/11 [==============================] - 1s 79ms/step - loss: 7454.9971
Epoch 178/2000
11/11 [==============================] - 1s 79ms/step - loss: 7425.5674
Epoch 179/2000
11/11 [==============================] - 1s 79ms/step - loss: 7396.4575
Epoch 180/2000
11/11 [==============================] - 16s 1s/step - loss: 7367.6929 - val_mrr: 0.2868 - val_mr: 218.7188 - val_hits@1: 0.1925 - val_hits@10: 0.4783 - val_hits@100: 0.7567
Epoch 181/2000
11/11 [==============================] - 1s 80ms/step - loss: 7339.1338
Epoch 182/2000
11/11 [==============================] - 1s 83ms/step - loss: 7310.8516
Epoch 183/2000
11/11 [==============================] - 1s 82ms/step - loss: 7282.8994
Epoch 184/2000
11/11 [=====================

In [7]:
X_test = X['test']
X_test[:2]

array([['/m/08966',
        '/travel/travel_destination/climate./travel/travel_destination_monthly_climate/month',
        '/m/05lf_'],
       ['/m/01hww_',
        '/music/performance_role/regular_performances./music/group_membership/group',
        '/m/01q99h']], dtype=object)

In [8]:
# predict on new triples
model.predict(X_test[:2])

array([10.322721,  7.204272], dtype=float32)

In [9]:
# get embedding of entities
model.get_embeddings(['/m/08966','/m/05lf_'], embedding_type='entity')

array([[ 0.26887387,  0.27902964, -0.20528151, ...,  0.15810564,
        -0.23209587,  0.33689767],
       [ 0.17871626, -0.19304618, -0.24542606, ...,  0.20739488,
        -0.2235089 , -0.22911648]], dtype=float32)

In [10]:
# get the entity and relation mappings to emb matrix
ent_to_idx, rel_to_idx = model.get_hyperparameter_dict()
len(ent_to_idx), len(rel_to_idx)

(14505, 237)

In [11]:
# import the evaluate_performance API from compat module
from ampligraph.compat import evaluate_performance
ranks = evaluate_performance(X_test, model, filter_triples=filter, corrupt_side='s,o', verbose=True)

# import the evaluation metrics
from ampligraph.evaluation.metrics import mrr_score, hits_at_n_score, mr_score

print('MR:', mr_score(ranks))
print('MRR:', mrr_score(ranks))
print('hits@1:', hits_at_n_score(ranks, 1))
print('hits@10:', hits_at_n_score(ranks, 10))

206/206 [==============================] - 51s 248ms/step
MR: 208.67266855856738
MRR: 0.289697242341989
hits@1: 0.1933897641647911
hits@10: 0.48177414619825815


In [12]:
from ampligraph.utils import save_model
# save the model
save_model(model, 'backward_model')


The path backward_model already exists. This save operation will overwrite the model                 at the specified path.
WARNING - Found untraced functions such as _get_ranks while saving (showing 1 of 1). These functions will not be directly callable after loading.


In [13]:
from ampligraph.utils import restore_model

# restore saved models or checkpoints
res_model = restore_model('backward_model')

In [14]:
# import the evaluate_performance API from compat module
from ampligraph.compat import evaluate_performance
ranks = evaluate_performance(X_test, res_model, filter_triples=filter, corrupt_side='s,o', verbose=True)

# import the evaluation metrics
from ampligraph.evaluation.metrics import mrr_score, hits_at_n_score, mr_score

print('MR:', mr_score(ranks))
print('MRR:', mrr_score(ranks))
print('hits@1:', hits_at_n_score(ranks, 1))
print('hits@10:', hits_at_n_score(ranks, 10))

206/206 [==============================] - 50s 244ms/step
MR: 208.67266855856738
MRR: 0.289697242341989
hits@1: 0.1933897641647911
hits@10: 0.48177414619825815


# Discovery 

In [15]:
# for discovery, import the required apis from the corresponding modules
# They are designed to be backward compatible
from ampligraph.discovery import discover_facts

discover_facts(X['train'][:100], 
               res_model, 
               top_n=100, 
               strategy='entity_frequency', 
               max_candidates=100, 
               target_rel='/location/country/form_of_government', 
               seed=0)


(array([['/m/06w99h3', '/location/country/form_of_government', '/m/09nqf'],
        ['/m/0fvf9q', '/location/country/form_of_government',
         '/m/05b4l5x']], dtype=object),
 array([27.5, 47.5]))